In [5]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
import sys
import cv2  # OPENCV2
import cv2 as cv



import gym
import tensorflow as tf
import numpy as np
import random
from collections import deque
from gym import wrappers


CNN_INPUT_WIDTH = 80
CNN_INPUT_HEIGHT = 80
CNN_INPUT_DEPTH = 1
SERIES_LENGTH = 4

REWARD_COFF = 3.0

INITIAL_EPSILON = 1.0
FINAL_EPSILON = 0.0001
REPLAY_SIZE = 50000
BATCH_SIZE = 32
GAMMA = 0.99
OBSERVE_TIME = 500
ENV_NAME = 'Breakout-v0'
EPISODE = 100000
STEP  = 1500
TEST = 10


class ImageProcess():
    def ColorMat2B(self, state):   # this is the function used for the game flappy bird
        height = 80
        width = 80
        state_gray = cv2.cvtColor( cv2.resize( state, ( height, width ) ) , cv2.COLOR_BGR2GRAY )
        _,state_binary = cv2.threshold( state_gray, 5, 255, cv2.THRESH_BINARY )
        state_binarySmall = cv2.resize( state_binary, ( width, height ))
        cnn_inputImage = state_binarySmall.reshape( ( height, width ) )
        return cnn_inputImage

    def ColorMat2Binary(self, state):
        # state_output = tf.image.rgb_to_grayscale(state_input)
        # state_output = tf.image.crop_to_bounding_box(state_output, 34, 0, 160, 160)
        # state_output = tf.image.resize_images(state_output, 80, 80, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        # state_output = tf.squeeze(state_output)
        # return state_output

        height = state.shape[0]
        width = state.shape[1]
        nchannel = state.shape[2]

        sHeight = int(height * 0.5)
        sWidth = CNN_INPUT_WIDTH

        state_gray = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)
        # print state_gray.shape
        # cv2.imshow('test2', state_gray)
        # cv2.waitKey(0)

        _, state_binary = cv2.threshold(state_gray, 5, 255, cv2.THRESH_BINARY)

        state_binarySmall = cv2.resize(state_binary, (sWidth, sHeight), interpolation=cv2.INTER_AREA)

        cnn_inputImg = state_binarySmall[25:, :]
        # rstArray = state_graySmall.reshape(sWidth * sHeight)
        cnn_inputImg = cnn_inputImg.reshape((CNN_INPUT_WIDTH, CNN_INPUT_HEIGHT))
        # print cnn_inputImg.shape

        return cnn_inputImg

    def ShowImageFromNdarray(self, state, p):
        imgs = np.ndarray(shape=(4, 80, 80))

        for i in range(0, 80):
            for j in range(0, 80):
                for k in range(0, 4):
                    imgs[k][i][j] = state[i][j][k]

        cv2.imshow(str(p + 1), imgs[0])
        cv2.imshow(str(p + 2), imgs[1])
        cv2.imshow(str(p + 3), imgs[2])
        cv2.imshow(str(p + 4), imgs[3])


class DQN():
    def __init__(self, env):
        self.imageProcess = ImageProcess()
        self.epsilon = INITIAL_EPSILON
        self.replay_buffer = deque()
        self.recent_history_queue = deque()
        self.action_dim = env.action_space.n
        self.state_dim = CNN_INPUT_HEIGHT * CNN_INPUT_WIDTH
        self.time_step = 0

        self.session = tf.InteractiveSession()
        self.create_network()
        # self.create_training_method()
        self.observe_time = 0

        self.merged = tf.summary.merge_all()
        self.summary_writer = tf.summary.FileWriter('./path', self.session.graph)

        self.session.run(tf.initialize_all_variables())

    def create_network(self):

        INPUT_DEPTH = SERIES_LENGTH

        self.input_layer = tf.placeholder(tf.float32, [None, CNN_INPUT_WIDTH, CNN_INPUT_HEIGHT, INPUT_DEPTH],
                                          name='status-input')
        self.action_input = tf.placeholder(tf.float32, [None, self.action_dim])
        self.y_input = tf.placeholder(tf.float32, [None])

        W1 = self.get_weights([8, 8, 4, 32])
        b1 = self.get_bias([32])

        h_conv1 = tf.nn.relu(tf.nn.conv2d(self.input_layer, W1, strides=[1, 4, 4, 1], padding='SAME') + b1)
        conv1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

        W2 = self.get_weights([4, 4, 32, 64])
        b2 = self.get_bias([64])

        h_conv2 = tf.nn.relu(tf.nn.conv2d(conv1, W2, strides=[1, 2, 2, 1], padding='SAME') + b2)
        # conv2 = tf.nn.max_pool( h_conv2, ksize = [ 1, 2, 2, 1 ], strides= [ 1, 2, 2, 1 ], padding= 'SAME' )

        W3 = self.get_weights([3, 3, 64, 64])
        b3 = self.get_bias([64])

        h_conv3 = tf.nn.relu(tf.nn.conv2d(h_conv2, W3, strides=[1, 1, 1, 1], padding='SAME') + b3)
        # conv3 = tf.nn.max_pool( h_conv3, ksize= [ 1,2,2,1], strides=[ 1,2,2,1 ],padding= 'SAME' )




        W_fc1 = self.get_weights([1600, 512])
        b_fc1 = self.get_bias([512])

        # h_conv2_flat = tf.reshape( h_conv2, [ -1, 11 * 11 * 32 ] )
        conv3_flat = tf.reshape(h_conv3, [-1, 1600])

        h_fc1 = tf.nn.relu(tf.matmul(conv3_flat, W_fc1) + b_fc1)

        W_fc2 = self.get_weights([512, self.action_dim])
        b_fc2 = self.get_bias([self.action_dim])

        self.Q_value = tf.matmul(h_fc1, W_fc2) + b_fc2
        Q_action = tf.reduce_sum(tf.multiply(self.Q_value, self.action_input), reduction_indices=1)
        self.cost = tf.reduce_mean(tf.square(self.y_input - Q_action))

        self.optimizer = tf.train.AdamOptimizer(1e-6).minimize(self.cost)

    # def create_training_method(self):
    #
    #   # if len(self.recent_history_queue) > 4:
    #   #   sess = tf.Session()
    #   #   print sess.run(self.Q_value)
    #   # global_step = tf.Variable(0, name='global_step', trainable=True)
    #   # self.optimizer = tf.train.AdamOptimizer( 0.001 ).minimize( self.cost )

    def train_network(self):
        self.time_step += 1

        minibatch = random.sample(self.replay_buffer, BATCH_SIZE)
        state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        next_state_batch = [data[3] for data in minibatch]
        done_batch = [data[4] for data in minibatch]
        # self.imageProcess.ShowImageFromNdarray( state_batch[0], 1 )

        y_batch = []
        Q_value_batch = self.Q_value.eval(feed_dict={self.input_layer: next_state_batch})

        # print Q_value_batch
        # print self.time_step
        # cv2.waitKey(0)

        # print Q_value_batch.shape

        for i in range(BATCH_SIZE):

            if done_batch[i]:
                y_batch.append(reward_batch[i])
            else:
                y_batch.append(reward_batch[i] + GAMMA * np.max(Q_value_batch[i]))

        self.optimizer.run(feed_dict={

            self.input_layer: state_batch,
            self.action_input: action_batch,
            self.y_input: y_batch

        })




    def percieve(self, state_shadow, action_index, reward, state_shadow_next, done, episode):

        action = np.zeros( self.action_dim )
        action[ action_index ] = 1

        self.replay_buffer.append([state_shadow, action, reward, state_shadow_next, done])

        self.observe_time += 1
        if self.observe_time % 1000 and self.observe_time <= OBSERVE_TIME == 0:
            print(self.observe_time)

        if len(self.replay_buffer) > REPLAY_SIZE:
            self.replay_buffer.popleft()

        if len(self.replay_buffer) > BATCH_SIZE and self.observe_time > OBSERVE_TIME:
            self.train_network()

    def get_greedy_action(self, state_shadow):

        rst = self.Q_value.eval(feed_dict={self.input_layer: [state_shadow]})[0]
        # print rst
        print(np.max( rst ))
        return np.argmax(rst)

    def get_action(self, state_shadow):
        if self.epsilon >= FINAL_EPSILON and self.observe_time > OBSERVE_TIME:
            self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000

        action = np.zeros(self.action_dim)
        action_index = None
        if random.random() < self.epsilon:
            action_index = random.randint(0, self.action_dim - 1)
        else:
            action_index = self.get_greedy_action(state_shadow)

        return action_index


    def get_weights(self, shape):
        weight = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(weight)

    def get_bias(self, shape):
        bias = tf.constant(0.01, shape=shape)
        return tf.Variable(bias)


def main():
    env = gym.make(ENV_NAME)
    env = wrappers.Monitor(env,'/home/users/cc/wkspc/artari/video',force=True)
    state_shadow = None
    next_state_shadow = None


    agent = DQN(env)
    total_reward_decade = 0

    # game_state = game.GameState()
    for episode in range(EPISODE):

        total_reward = 0
        state = env.reset()
        state = agent.imageProcess.ColorMat2Binary(state)  # now state is a binary image of 80 * 80
        state_shadow = np.stack((state, state, state, state), axis=2)

        for step in range(STEP):
#             env.render(mode='rgb_array')
            action = agent.get_action(state_shadow)

            next_state, reward, done, _ = env.step(action)

            next_state = np.reshape( agent.imageProcess.ColorMat2Binary( next_state ), ( 80,80,1 ) )

            # print next_state.shape
            # print state_shadow.shape
            next_state_shadow = np.append( next_state, state_shadow[ :,:,:3 ], axis= 2 )

            total_reward += reward
            agent.percieve(state_shadow, action, reward, next_state_shadow, done, episode)
            state_shadow = next_state_shadow

            if done:
                break
        with open('./cored1.txt', 'a', encoding='utf-8') as corefile:
            corefile.writelines(['Episode:',str(episode), ' | Total Point this Episode is:',str(total_reward),'\n'])
        print('Episode:', episode, 'Total Point this Episode is:', total_reward)
        total_reward_decade += total_reward
        if episode % 10 == 0:
            print('-------------')
            print('Decade:', episode / 10, 'Total Reward in this Decade is:', total_reward_decade)
            print('-------------')
            total_reward_decade = 0



if __name__ == '__main__':
    main()

/home/amax/soft/gym/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


Episode: 0 Total Point this Episode is: 3.0
-------------
Decade: 0.0 Total Reward in this Decade is: 3.0
-------------
Episode: 1 Total Point this Episode is: 1.0
0.03163645
0.03339512
0.057693142
Episode: 2 Total Point this Episode is: 2.0
0.06849468
0.070969336
0.08625228
0.085552566
0.08910237
0.093955405
Episode: 3 Total Point this Episode is: 0.0
0.09969205
0.100794435
0.112541415
0.1134626
0.11893503
0.12524754
0.12491855
0.12619863
0.12693809
0.12932725
0.1437407
0.1555936
Episode: 4 Total Point this Episode is: 0.0
0.16611177
0.16731364
0.17287141
0.17247748
0.18464354
0.18867053
0.19375943
0.2125733
0.21561742
0.22829527
0.23095393
0.23970453
0.24677305
0.24613135
0.2530135
0.26005724
0.27468053
0.2786535
0.27611127
Episode: 5 Total Point this Episode is: 2.0
0.31851396
0.32080427
0.32450286
0.3250908
0.3401696
0.3428564
0.34633392
0.3540193
0.38259187
0.39041883
0.39372522
0.40029067
0.40243742
0.4046895
0.40902138
0.4160513
0.41697386
0.4188615
0.41670284
0.41704363
0.41422

0.77808005
0.7781541
0.7786241
0.7788062
0.77873826
0.7799171
0.78070134
0.780153
0.7832092
0.7854706
0.78453225
0.7926573
0.79294664
0.7917959
0.7911219
0.7904106
0.792674
0.7895571
0.7896987
0.79080254
0.7894096
0.79015154
0.7880912
0.7896615
0.78496814
0.7886972
0.7865725
0.7896693
0.7909135
0.7914623
0.7916372
0.7930036
0.79111433
0.79347384
0.79241306
0.791043
0.7920083
0.7913794
0.79016376
0.7909032
0.7910938
0.7908165
0.79289705
0.7931673
0.79535186
0.801314
0.800569
0.80013543
0.8055464
0.8075958
0.8075548
0.7978616
0.79844034
Episode: 17 Total Point this Episode is: 0.0
0.8132323
0.8137484
0.8154867
0.813425
0.81171954
0.8067288
0.8076466
0.8091108
0.8135771
0.8119958
0.8073698
0.80533457
0.8035765
0.80265975
0.80302745
0.80411273
0.80017936
0.7961115
0.79363084
0.78934157
0.78963184
0.7894663
0.7895334
0.78729725
0.780081
0.7790466
0.778024
0.7769106
0.7746977
0.7738771
0.7720046
0.7706617
0.7693282
0.76736516
0.76667017
0.7649766
0.76548743
0.76713556
0.7635359
0.7619147
0.7

0.81993425
0.8210064
0.82307744
0.8238333
0.8254236
0.8277544
0.82692015
0.8269917
0.8283776
0.82822573
0.82758695
0.82948285
0.82836163
0.8310467
0.8310264
0.8306979
0.8331746
0.83306843
0.83384037
0.83408034
0.83678615
0.84051174
0.8403362
0.8417465
0.84279454
0.8430215
0.84144443
0.83755976
0.8374809
0.8371885
0.83462715
0.8339174
0.83323205
0.8328588
0.8311619
0.82933587
0.8273796
0.82644045
0.82733315
0.82742137
0.8288825
0.83075684
0.83156127
0.83304256
0.83023936
0.83087695
0.83147013
0.82824004
0.8301755
0.83157206
0.8313579
0.8306081
0.82988983
0.82957494
0.83027464
0.8298109
0.8316223
0.8347148
0.832777
0.8280924
0.8258065
0.8203951
0.8159064
0.8159301
0.8160161
0.812768
0.8102603
0.8117388
0.81427985
0.8135184
0.8131867
0.81329525
0.81207967
0.8109647
0.8109947
0.81183857
0.80867946
0.8082321
0.80659795
0.8053033
0.80467254
0.80244446
0.8022675
0.80203277
0.8017672
0.8014756
0.80126786
0.8004015
0.80054927
0.80266887
0.8031913
0.8030243
0.8018935
0.80050457
0.7969891
0.79751

0.8416306
0.8454213
0.8389279
0.84073454
0.82893294
0.82971406
0.82529646
0.8287245
0.8296511
0.8302168
0.82970214
0.8276283
0.826793
0.82722676
0.82508546
0.82582366
0.82728744
0.82590115
0.8267398
0.826135
0.82764477
0.8266295
0.8257435
0.8283809
0.82385784
0.82415754
0.8242535
0.82469255
0.82323116
0.82288486
0.8223331
0.81708705
0.8174165
0.8147533
0.81576747
0.8161585
0.8154327
0.81536657
0.81517875
0.81497175
0.8145145
0.8140449
0.8135666
0.8130777
0.8123474
0.8089721
0.8065657
0.8054504
0.8043797
0.80334496
0.8025866
0.8019858
0.799861
0.801199
0.8018821
0.8014387
0.7994346
0.8002931
0.80096835
0.8013106
0.8033041
0.80570453
0.8077327
0.80613273
0.80426127
0.80667967
0.80653673
0.80775243
0.81034523
0.8103302
0.81270134
0.8119826
0.8122587
0.8152598
0.8153387
0.81771684
0.8188687
0.8196173
0.822473
0.8250749
0.8277595
0.8305672
0.83715904
0.8438536
0.841607
0.8443589
0.8462655
0.85547245
0.8594031
0.86179864
0.86257774
0.8646736
0.8703218
0.87108016
0.8710129
0.8714564
0.8710564

...

1.4432789
1.4372324
1.4290221
1.4252963
1.4249226
1.4292121
1.4349177
1.4351339
1.4390025
1.4451945
1.444
1.4457757
1.445344
1.444948
1.4453409
1.4457692
1.4460253
1.4462041
1.4464301
1.4470638
1.4482744
1.4497212
1.4705392
1.4810729
1.4968656
1.5095417
1.489882
1.4582909
1.471973
1.4645681
1.4726373
1.4732988
1.4825518
1.4837784
1.4909153
1.4858954
1.477534
1.4755354
1.4751189
1.4750693
1.5065283
1.5291624
1.5311123
1.5432788
1.5570015
1.5637546
1.5607781
1.5714831
1.5733917
1.5764004
1.5779498
1.5795072
1.5809268
1.582772
1.5841488
1.5941164
1.5973469
1.6014569
1.603226
1.5952891
1.5890758
1.5882138
1.5839336
1.5772278
1.5737306
1.5741694
1.5694468
1.5725625
1.576011
1.5710603
1.5724177
1.5688336
1.5694772
1.5683098
1.5803505
1.5807397
1.5813674
1.5834664
1.582672
1.5801656
1.5795463
1.5717177
1.5736668
1.5616087
1.5609493
1.5607904
1.5567733
1.564584
1.5764353
1.5797439
1.5817688
1.5863043
1.5959224
1.5919648
1.6037498
1.6005404
1.6064239
1.6112878
1.611025
1.6306667
1.6638422
1.726

1.6458138
1.6434028
1.6410472
1.6391972
1.6373084
1.6361284
1.6349096
1.6334506
1.6320294
1.6307112
1.6291258
1.6272693
1.6244124
1.6215341
1.6188008
1.6156971
1.612408
1.6093229
1.6074127
1.6055063
1.6035987
1.601429
1.5994897
1.5972004
1.5947914
1.5923972
1.5901709
1.5872915
1.5842963
1.5811133
1.5788921
1.577321
1.5755936
1.5736737
1.5719274
1.5699455
1.5674874
1.5649811
1.5624009
1.5600436
1.5560765
1.5523678
1.5493131
1.5472528
1.5452718
1.5431583
1.5408962
1.5385251
1.5360714
1.5333747
1.5291309
1.5252293
1.5214714
1.5179386
1.5152087
1.512209
1.5096335
1.5078002
1.5055509
1.5038588
1.5020683
1.5002533
1.4984558
1.4966815
1.4950725
1.4926074
1.4908986
1.4904987
1.4896992
1.488455
1.4872569
1.4866929
1.4858787
1.4849815
1.4839929
1.4838481
1.4835775
1.4832699
1.4828241
1.4821584
1.4815767
1.4806247
1.4803247
1.479605
1.4784156
1.4771283
1.4756534
1.4739606
1.4717059
1.4702592
1.4686955
1.467352
1.4648219
1.4621506
1.459462
1.4569466
1.4545975
1.4523154
1.4500486
1.4480866
1.446117

1.3269452
1.3636248
1.4492998
1.5269042
1.6532931
1.8363663
1.7941226
1.7625074
1.7822385
1.672725
1.4744103
1.4884869
1.5514219
1.5051974
1.5584064
1.5232687
1.4791374
1.4262884
1.3744966
1.3658749
1.3238807
1.3057088
1.300076
1.2989396
1.3022666
1.3002754
1.2976743
1.2948432
1.2941899
1.2907746
1.2950581
1.2905587
1.2831305
1.2743484
1.2693313
1.2712674
1.277879
1.2826304
1.2884738
1.2887355
1.2936833
1.2930218
1.2923065
1.291968
1.2931883
1.2950695
1.2968811
1.2967656
1.2966195
1.2962223
1.2962162
1.2947689
1.2936945
1.29223
1.2912955
1.2894648
1.288566
1.2891247
1.289499
1.2906697
1.2917082
1.2926944
1.2936058
1.2944735
1.2955978
1.2965862
1.299497
1.3023074
1.3050355
1.3072475
1.3094122
1.3120466
1.3147595
1.3172119
1.3195305
1.3218911
1.3232863
1.3244219
1.3259816
1.3273132
1.3284631
1.3283999
1.3273889
1.3264514
1.324793
1.3234098
1.3227847
1.3222537
1.321646
1.3209337
1.3201922
1.3185561
1.3168497
1.3152211
1.3137726
1.3121654
1.3111979
1.3101968
1.3091079
1.3079038
1.3065819
1

1.3885928
1.3968581
1.4135424
1.4189789
1.4258429
1.4221284
1.4368961
1.4379297
1.4366072
1.4366732
1.4364446
1.4360434
1.4361062
1.4357879
1.4353775
1.4348109
1.4340006
1.4331354
1.4320924
1.4309561
1.4298393
1.4293513
1.4292978
1.4288058
1.4287043
1.4282782
1.4275101
1.4265871
1.4256141
1.4246341
1.4226195
1.4202062
1.4179318
1.4157383
1.413608
1.4114342
1.4089819
1.406752
1.4048107
1.4032371
1.4017311
1.3991512
1.3974137
1.3975114
1.398015
1.3982518
1.3981179
1.3977803
1.3973163
1.3963898
1.3958192
1.3959419
1.3955654
1.3950942
1.395069
1.3954476
1.395678
1.3966361
1.3973124
1.39772
1.3968582
1.3961104
1.3950657
1.3941356
1.3937546
1.3941056
1.3946666
1.3959324
1.3980391
1.4001987
1.4023587
1.4052391
1.4088136
1.4122645
1.4161963
1.4198539
1.423565
1.4274192
1.43171
1.4363452
1.4410455
1.4462321
1.4645877
1.4604121
1.4640387
1.4622034
1.4693618
1.4791486
1.4747634
1.4773184
1.4836023
1.4874634
1.4941362
1.4967473
1.4937803
1.4951708
1.4979308
1.4962056
1.5002023
1.5053272
1.5134774


Error: Tried to reset environment which is not done. While the monitor is active for Breakout-v0, you cannot call reset() unless the episode is over.

In [7]:
import gym
from gym.wrappers import Monitor
import itertools
import numpy as np
import os
import random
import sys
import tensorflow as tf
if "../" not in sys.path:
  sys.path.append("../")
from lib import plotting
from collections import deque, namedtuple
env = gym.envs.make("Breakout-v0")
# Atari Actions: 0 (noop), 1 (fire), 2 (left) and 3 (right) are valid actions
VALID_ACTIONS = [0, 1, 2, 3]

ModuleNotFoundError: No module named 'lib'